In [1]:
import requests
import json
import datetime
import pandas as pd

In [2]:
def r_to_j(r):
    myJ_s = ''
    line_num=1
    for n in r:
        #print(line_num, n)
        myJ_s += n.decode("'iso-8859-1'")
        
#    print(myJ_s)
    myJ = json.loads(myJ_s)
#    print(json.dumps(myJ_s, indent=1))
    return myJ


In [3]:
def do_r(req, page_size=2000, page_num=0,debug=False, attrs=[]):
    myJ = {}
    rJ = {'count': 0, 'g' : {}}
    keepGoing=1
    gDebug=0

    ctr=1
    
    req += '&page_size='+str(page_size)+'&page_num='

    while keepGoing:
        keepGoing=1
        page_num += 1
        if page_num > 599:
            keepGoing = 0

        ctr += 1
        if ctr >= 100:
            print ('100th request', datetime.datetime.now())
            ctr=1
            
        r = requests.get(req + str(page_num) )
        
        if debug:
            print(r, page_num, datetime.datetime.now())
        
        myJ = r_to_j(r)
        
        if debug:
            print('myj=', json.dumps(myJ, indent=1))
            debug=0
        
        if len(myJ['items']) == 0:
            keepGoing=0
            continue

        rJ['count']+= len(myJ['items'])
        outJ={}
        for j in myJ['items']:
            if gDebug:
                print (json.dumps(j['umm']))
                if 'GranuleUR' in j['umm']:
                    print('****************--->', j)
                else:
                    print ('producer_granule_id NOT FOUND!', j)
                gDebug=0
            outJ={}
            for a in attrs:
                if a in j['umm']:
                    outJ[a]=j['umm'][a]
                
            rJ['g'][j['umm']['GranuleUR']]=outJ

    return rJ

In [4]:
# Get a list of short_names from a provider
import time

my_provider='ASF'
my_shortNames=[]
#print (r.headers)
myJ_s = ''
collectionsJ={}
uniqueKeysJ={}

page_num=0
keepGoing=1
debug=0
while keepGoing:
    
    page_num += 1
    if page_num > 200:
        keepGoing = 0
    print ('before request.get')  
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?platform=AQUA&sort_key=short_name&page_size=2&page_num='+str(page_num))
    r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?provider=ASF&sort_key=short_name&page_size=2000&page_num='+str(page_num))
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?instrument=MODIS&INSTRUMENT=MODIS&provider=LPDAAC_ECS&sort_key=short_name&page_size=2&page_num='+str(page_num))
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?granule_data_format=HDF-EOS2&sort_key=short_name&page_size=2000&page_num='+str(page_num))
    print ('after request.get')  

    myJ_s = ''
    line_num=1
    for n in r:
        #print(line_num, n)
        myJ_s += n.decode("'iso-8859-1'")
        
    #print(myJ_s)
    myJ = json.loads(myJ_s)
    #print(json.dumps(myJ, indent=1))

    if len(myJ['items']) == 0:
        keepGoing=0
        continue
    
    for r in myJ['items']:
        j=r['meta']
        #print ('j=', j)
        if debug == 1:
            print ('r=', json.dumps(r,indent=1))
            debug=0
        #for k in j:
        #    print ('meta',k, j[k])
        j=r['umm']
#        for k in j:
            #print ('umm', k)
#            if k == 'ShortName':
        my_shortNames.append({'ShortName' : j['ShortName'],
                                     'native-id' : r['meta']['native-id'],
                                'EntryTitle' : j['EntryTitle'],
                              'Count':r['meta']['granule-count'] if 'granule-count' in r['meta'] else 0,
                             'Format' : j ['ArchiveAndDistributionInformation'] if 'ArchiveAndDistributionInformation' in j else 'UNKNOWN'
                             })

i=1
for s in my_shortNames:
    print (i, s)
    i+=1
#print (my_shortNames)

before request.get
after request.get
before request.get
after request.get
1 {'ShortName': 'AIRSAR_INT_JPG', 'native-id': 'AIRSAR_ALONGTRACK_INTERFEROMETRY_JPG', 'EntryTitle': 'AIRSAR_ALONGTRACK_INTERFEROMETRY_JPG', 'Count': 0, 'Format': 'UNKNOWN'}
2 {'ShortName': 'AIRSAR_POL_3FP', 'native-id': 'AIRSAR_POLSAR_3_FREQ_POLARIMETRY', 'EntryTitle': 'AIRSAR_POLSAR_3_FREQ_POLARIMETRY', 'Count': 0, 'Format': 'UNKNOWN'}
3 {'ShortName': 'AIRSAR_POL_SYN_3FP', 'native-id': 'AIRSAR_POLSAR_SYNOPTIC_3_FREQ_POLARIMETRY', 'EntryTitle': 'AIRSAR_POLSAR_SYNOPTIC_3_FREQ_POLARIMETRY', 'Count': 0, 'Format': 'UNKNOWN'}
4 {'ShortName': 'AIRSAR_TOP_C-DEM_STOKES', 'native-id': 'AIRSAR_TOPSAR_C-BAND_DEM_AND_STOKES', 'EntryTitle': 'AIRSAR_TOPSAR_C-BAND_DEM_AND_STOKES', 'Count': 0, 'Format': 'UNKNOWN'}
5 {'ShortName': 'AIRSAR_TOP_DEM', 'native-id': 'AIRSAR_TOPSAR_DEM', 'EntryTitle': 'AIRSAR_TOPSAR_DEM', 'Count': 0, 'Format': 'UNKNOWN'}
6 {'ShortName': 'AIRSAR_TOP_DEM_C', 'native-id': 'AIRSAR_TOPSAR_DEM_C', 'EntryTit

In [ ]:
r_base = 'https://cmr.earthdata.nasa.gov/search/granules.umm_json?short_name='
i=1


file=open('cmr_spatial.tsv','w')

for s in my_shortNames:
    print (s)
   # if s['ShortName']=='GPM_2AGPROFNOAA20ATMS_CLIM':
   #     pass
   # else:
   #     continue

    
# {'ShortName': 'AA_L2A', 
#  'native-id': 'AMSR/ADEOS-II L2A Global Swath Spatially-Resampled Brightness Temperatures V001', 
#  'EntryTitle': 'AMSR/ADEOS-II L2A Global Swath Spatially-Resampled Brightness Temperatures V001',
#  'Count': 0, 
#  'Format': {'FileDistributionInformation': [{'FormatType': 'Native', 'Format': 'HDF-EOS', 'FormatDescription': 'FTP 68 MB'}]}}
    
    r = r_base + s['ShortName']
    r += '&temporal=2017-01-01T00:00:00Z,2021-06-18T23:59:59Z'
    
    print (r)

    gJ = do_r(r, attrs=['GranuleUR','DataGranule', 'CollectionReference', 'TemporalExtent'],debug=False)
    
    if gJ['count'] == 0:
        continue
        
    print ('# Granules', gJ['count'])
    #for g in gJ['g']:
    #    print ('Example G', json.dumps(gJ['g'][g]))
    #    break
    
    for g in gJ['g']:
        gn+=1
        gm=gJ['g'][g]
        #print(g, json.dumps(gJ['g'][g], indent=1))
        


        x_out = [
            s['ShortName'],
            "", # No VersionID for ASF
            g, #GranuleUR
            str(gm['DataGranule']['ArchiveAndDistributionInformation'][0]['Size']),
            gm['DataGranule']['ArchiveAndDistributionInformation'][0]['SizeUnit'],
            str(gm['CollectionReference']),
            gm['TemporalExtent']['RangeDateTime']['BeginningDateTime'],
            gm['TemporalExtent']['RangeDateTime']['EndingDateTime'],
            myMP_WKT
        ]
        
        x_line = '~'.join(x_out)
        x_line += '\n'
        file.write(x_line)
        #print (x_line)
        #break
        

    if gn > 0:
        print (granuleSizeTotal)


#for i in cmr_granule_table:
#    print (i)
#    break

file.close()


{'ShortName': 'AIRSAR_INT_JPG', 'native-id': 'AIRSAR_ALONGTRACK_INTERFEROMETRY_JPG', 'EntryTitle': 'AIRSAR_ALONGTRACK_INTERFEROMETRY_JPG', 'Count': 0, 'Format': 'UNKNOWN'}
https://cmr.earthdata.nasa.gov/search/granules.umm_json?short_name=AIRSAR_INT_JPG&temporal=2019-01-01T00:00:00Z,2021-05-15T23:59:59Z
{'ShortName': 'AIRSAR_POL_3FP', 'native-id': 'AIRSAR_POLSAR_3_FREQ_POLARIMETRY', 'EntryTitle': 'AIRSAR_POLSAR_3_FREQ_POLARIMETRY', 'Count': 0, 'Format': 'UNKNOWN'}
https://cmr.earthdata.nasa.gov/search/granules.umm_json?short_name=AIRSAR_POL_3FP&temporal=2019-01-01T00:00:00Z,2021-05-15T23:59:59Z
{'ShortName': 'AIRSAR_POL_SYN_3FP', 'native-id': 'AIRSAR_POLSAR_SYNOPTIC_3_FREQ_POLARIMETRY', 'EntryTitle': 'AIRSAR_POLSAR_SYNOPTIC_3_FREQ_POLARIMETRY', 'Count': 0, 'Format': 'UNKNOWN'}
https://cmr.earthdata.nasa.gov/search/granules.umm_json?short_name=AIRSAR_POL_SYN_3FP&temporal=2019-01-01T00:00:00Z,2021-05-15T23:59:59Z
{'ShortName': 'AIRSAR_TOP_C-DEM_STOKES', 'native-id': 'AIRSAR_TOPSAR_C-BAND

NameError: name 'cmr_granule_table' is not defined

In [6]:
df_cmr=pd.DataFrame(cmr_granule_table, columns=['GranuleUR', 'DataGranule', 'CollectionReference', 'TemporalExtent', 'SpatialExtent'])
df_cmr
#df_cmr['start_ts']=pd.to_datetime(df_cmr['start'],format='%Y%m%dT%H:%M:%S.%f').dt.time
#df_cmr['end_ts']  =pd.to_datetime(df_cmr['end'],format='%Y%m%dT%H:%M:%S.%f').dt.time

#df['Open'] =       pd.to_datetime(df    ['OPEN '],format= '%H:%M:%S' ).dt.time

,GranuleUR,DataGranule,CollectionReference,TemporalExtent,SpatialExtent
0,S1-GUNW-A-R-166-tops-20200101_20191208-014056-...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...",{'EntryTitle': 'Sentinel-1 Interferograms (BET...,{'RangeDateTime': {'BeginningDateTime': '2020-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
1,S1-GUNW-A-R-166-tops-20200101_20191220-014056-...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...",{'EntryTitle': 'Sentinel-1 Interferograms (BET...,{'RangeDateTime': {'BeginningDateTime': '2020-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
2,S1-GUNW-A-R-166-tops-20200101_20181225-014056-...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...",{'EntryTitle': 'Sentinel-1 Interferograms (BET...,{'RangeDateTime': {'BeginningDateTime': '2020-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
3,S1-GUNW-A-R-166-tops-20200101_20191220-014056-...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...",{'EntryTitle': 'Sentinel-1 Interferograms (BET...,{'RangeDateTime': {'BeginningDateTime': '2020-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
4,S1-GUNW-A-R-166-tops-20200101_20191208-014056-...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...",{'EntryTitle': 'Sentinel-1 Interferograms (BET...,{'RangeDateTime': {'BeginningDateTime': '2020-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
...,...,...,...,...,...
16341,UA_SanAnd_23517_18080-011_20023-009_0668d_s01_...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...",{'EntryTitle': 'UAVSAR_INSAR_METADATA'},{'RangeDateTime': {'BeginningDateTime': '2018-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
16342,UA_SanAnd_26524_19002-011_20026-015_0575d_s01_...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...",{'EntryTitle': 'UAVSAR_INSAR_METADATA'},{'RangeDateTime': {'BeginningDateTime': '2019-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
16343,UA_SanAnd_23029_19030-019_20024-001_0469d_s01_...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...",{'EntryTitle': 'UAVSAR_INSAR_METADATA'},{'RangeDateTime': {'BeginningDateTime': '2019-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
16344,UA_lowman_23205_19084-017_20002-007_0042d_s01_...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...",{'EntryTitle': 'UAVSAR_INSAR_METADATA'},{'RangeDateTime': {'BeginningDateTime': '2019-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...


In [7]:
df_cmr.to_csv('cmr_spatial.tsv', sep='~', index=False)

CREATE EXTERNAL TABLE `cmr_3`(
 GranuleUR string,
 DataGranule string,
 CollectionReference string,
 TemporalExtent string,
 SpatialExtent string
)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES ( 
  'quoteChar'='\u0000', 
  'separatorChar'='~') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://nasa-ems-sandbox/ems_analysis/ad-hoc/analyst1/cmr_3'
TBLPROPERTIES (
  'skip.header.line.count'='1', 
  'transient_lastDdlTime'='1620856378')

In [12]:
import sys
!{sys.executable} -m pip install PyAthena
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from pyathena import connect
from pyathena.pandas.util import to_sql

conn = connect(s3_staging_dir='s3://esdis-ems-athena', region_name='us-west-2')